### This script shows how to use the existing code in opengrid to create a baseload electricity consumption benchmark. 

In [ ]:
import os, sys
import inspect
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import HourLocator, DateFormatter, AutoDateLocator
import datetime as dt
import pytz
import pandas as pd
import pdb
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# add the path to opengrid to sys.path
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))
from opengrid.library import config
c = config.Config()

sys.path.append(c.get('tmpo', 'folder'))

try:
    if os.path.exists(c.get('tmpo', 'data')):
        path_to_tmpo_data = c.get('tmpo', 'data')
except:
    path_to_tmpo_data = None

from opengrid.library.houseprint import houseprint

%matplotlib inline
plt.rcParams['figure.figsize']=14,8

### We create one big dataframe, the columns are the sensors of type *gas*

In [ ]:
hp = houseprint.load_houseprint_from_file('new_houseprint.pkl')
hp.init_tmpo(path_to_tmpo_data=path_to_tmpo_data)

In [ ]:
df = hp.get_data(sensortype='gas')
df = df.diff()
df = df[df>0]
df = df*10*60/1000 #from L/min to kW: 10 Wh/L, 60 min/h, 1 kW/1000W

In [ ]:
# plot timeseries and load duration for each retained sensor
for sensor in df.columns:
    plt.figure()
    ax1=plt.subplot(121)
    plt.plot_date(df.index, df[sensor], '-', label=sensor)
    plt.ylabel('kW')
    plt.legend()
    
    ax2=plt.subplot(122)
    plt.plot(np.sort(df[sensor])[::-1], label=sensor)
    plt.ylabel('kW')
    plt.legend()